# SF-DST Car Price Prediction AnnaCoonDa
# Предсказание стоимости авто
# Парсинг данных для тестового датасета - долгий вариант
### Внимание: сбор информации идёт без распараллеливания, код работает около 12 часов!
![](https://seo2you.ru/upload/static/img/74147.jpg)

# Импорт необходимых библиотек

In [1]:
import requests
import pandas as pd
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
from itertools import chain

# Определяем необходимые функции

In [2]:
#Количество страниц с объявлениями для каждой марки машины
def pages(link):
    driver = webdriver.Chrome()
    driver.get(link)
    page = BeautifulSoup(driver.page_source, 'html.parser')
    pages = page.find_all('span', class_='ControlGroup ControlGroup_responsive_no ControlGroup_size_s ListingPagination-module__pages')
    if pages == []:
        total_page_count = 1
    else:
        total_page_count = int(page.find_all('span', class_='Button__text')[-4].text)
    driver.close()
    time.sleep(0.3)
    return total_page_count

#Разбиение линков на страницы для каждой модели
def ads_pages(link):
    pages_for_car = []
    pages_cnt = pages(link)
    if pages_cnt == 1:
        pages_for_car.append(link)
    else:
        for i  in range(1, pages_cnt+1):
            link_page = link+'?page='+str(i)
            pages_for_car.append(link_page)
    return pages_for_car

#Получаем линки для каждого объявления
def get_ad_url(link):
    time.sleep(0.2)
    res = requests.get(link)
    page = BeautifulSoup(res.text, 'html.parser')
    ad_links = page.find_all('a', class_='Link ListingItemTitle-module__link')
    car_links = list(map(lambda x: x.get('href'), ad_links))
    
    return car_links

def car_data(link):
    time.sleep(0.1)
    res = requests.get(link)    
    res.encoding ='utf8'     #Это необходимо, т.к. в тексте много кириллицы
    page = BeautifulSoup(res.text, 'html.parser')
    car_dict = {}

    car_dict['car_url'] = link
    car_dict['parsing_unixtime'] = int(time.time())
    #Начинаем вынимать из кода необходимую информацию, в формате словарей
    # в позициях script ищем необходимые вхождения 
    for script in page.find_all("script"):
        if 'complectation":{"id"' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
    # в a  ищем 'complectation":' и содержимое между {}. Отсекаем начало, оставляя только содержимое словаря
            car_dict['complectation_dict'] = re.search(r'complectation":{"id.*?}', a)[0][15:]

        if 'equipment":{' in str(script):
            a = str(script)  
        # в a  ищем 'equipment":' и содержимое между {}. Отсекаем начало, оставляя только содержимое словаря
            car_dict['equipment_dict'] = re.search(r'equipment":{.*?}', a)[0][11:]

        if '{"mileage":' in str(script):
            a = str(script)  
        # в a  ищем '{"mileage":' и содержимое между {}. Отсекаем начало, оставляя только содержимое словаря
            car_dict['mileage'] = re.search(r'"mileage":\d*', a)[0][10:]

        if '"model_info":' in str(script):
            a = str(script)  
            car_dict['model_info'] = re.search(r'"model_info":{.*?}', a)[0][13:]
            car_dict['model_name'] = re.search(r'model_info":{"code":".*?"', a)[0][20:].strip('"')

        if 'super_gen":{' in str(script):
            a = str(script)  
            car_dict['super_gen'] = re.search(r'super_gen":{.*?}', a)[0][11:] 

        if 'vendor":"' in str(script):
            a = str(script)  
            car_dict['vendor'] = re.search(r'vendor":".*?"', a)[0][9:].strip('"')

    #По соответствующим тэгам вынимаем  в словарь необходимую информацию

    for tag in page.find_all('div'):
        if tag.get("title") == "Идентификатор объявления":
            car_dict['sell_id'] = re.search(r'\d+', tag.text)[0]


    for tag in page.find_all("meta"):
        if tag.get("itemprop") == "bodyType":
            car_dict['bodyType'] = tag.get("content")

        if tag.get("itemprop") == "brand":
            car_dict['brand'] = tag.get("content")

        if tag.get("itemprop") == "color":
            car_dict['color'] = tag.get("content")

        if tag.get("itemprop") == "description":
            car_dict['description'] = tag.get("content")

        if tag.get("itemprop") == "engineDisplacement":
            car_dict['engineDisplacement'] = tag.get("content")

        if tag.get("itemprop") == "enginePower":
            car_dict['enginePower'] = tag.get("content")

        if tag.get("itemprop") == "fuelType":
            car_dict['fuelType'] = tag.get("content")

        if tag.get("itemprop") == "modelDate":
            car_dict['modelDate'] = tag.get("content")

        if tag.get("itemprop") == "name":
            car_dict['name'] = tag.get("content")

        if tag.get("itemprop") == "numberOfDoors":
            car_dict['numberOfDoors'] = tag.get("content")

        if tag.get("itemprop") == "price":
            car_dict['price'] = tag.get("content")

        if tag.get("itemprop") == "priceCurrency":
            car_dict['priceCurrency'] = tag.get("content")

        if tag.get("itemprop") == "productionDate":
            car_dict['productionDate'] = tag.get("content")

        if tag.get("itemprop") == "vehicleConfiguration":
            car_dict['vehicleConfiguration'] = tag.get("content")

        if tag.get("itemprop") == "vehicleTransmission":
            car_dict['vehicleTransmission'] = tag.get("content")


    span_CardInfoRow__cell = page.find_all('span', {'class': 'CardInfoRow__cell'})

    for i,tag in enumerate (span_CardInfoRow__cell):
        if tag.text == "Владельцы":
            car_dict['Владельцы'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел
        if tag.text == "Владение":
            car_dict['Владение'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') 
        if tag.text == "ПТС":
            car_dict['ПТС'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ')     

        if tag.text == "Привод":
            car_dict['Привод'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ')  

        if tag.text == "Руль":
            car_dict['Руль'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') 

        if tag.text == "Состояние":
            car_dict['Состояние'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ')  

        if tag.text == "Таможня":
            car_dict['Таможня'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') 
    return car_dict

# Получение данных для обучающей выборки
Распарсим сайт auto.ru. По условиям проекта, нас интересуют только подержанные авто без ограничения списка марок. При первичном лобовом парсинге обнаружилась следующая проблема. Несмотря на то, что  всего на сайте активно более 50 тысяч объявлений о продаже, прямой парсинг по фильтру "продажа подержанных автомобилей" позволяет получить только 38х99=3762 объявления. 
Поэтому ссылки будут получаться по следующей схеме:
1. Открывается сайт auto.ru
2. C помощью модуля selenium раскрывается список всех марок, по которым есть объявления о продаже (для этого программе необходимо кликнуть на ссылку).
3. Вынимаются ссылки на каждую марку, при этом параметр all обычной строковой заменой подменяется на used
   (логика проста: если марка есть в продаже в категории "все", то и категория "подержанные" может включать объявления)
4. Далее циклом прогоним список ссылок на продажу подержанных моделей каждой фирмы, вынем объявления и получим из них нужные     данные. Поскольку на странице одновременно отображается 38 объявлений, то будем учитывать пагинацию.

In [9]:
driver = webdriver.Chrome()
driver.get('https://auto.ru/moskovskaya_oblast')

#Кликаем на ссылку, чтобы раскрыть список всех марок
driver.find_element_by_class_name('IndexMarks__show-all').click()

In [10]:
#Ставим небольшую задержку, т.к. сайту нужно время, чтобы открыть ссылку
time.sleep(2)

#На странице изменился код, теперь можно получить полный список марок с объявлениями
soup = BeautifulSoup(driver.page_source, 'html.parser')
marks_to_sale= soup.find('div', class_='IndexMarks__marks-with-counts')
driver.close()
marks_to_sale

<div class="IndexMarks__marks-with-counts"><div class="IndexMarks__col"><a class="IndexMarks__item" href="https://auto.ru/moskovskaya_oblast/cars/ac/all/"><div class="IndexMarks__item-name">AC</div><div class="IndexMarks__item-count">3</div></a><a class="IndexMarks__item" href="https://auto.ru/moskovskaya_oblast/cars/amc/all/"><div class="IndexMarks__item-name">AMC</div><div class="IndexMarks__item-count">2</div></a><a class="IndexMarks__item" href="https://auto.ru/moskovskaya_oblast/cars/acura/all/"><div class="IndexMarks__item-name">Acura</div><div class="IndexMarks__item-count">56</div></a><a class="IndexMarks__item" href="https://auto.ru/moskovskaya_oblast/cars/alfa_romeo/all/"><div class="IndexMarks__item-name">Alfa Romeo</div><div class="IndexMarks__item-count">33</div></a><a class="IndexMarks__item" href="https://auto.ru/moskovskaya_oblast/cars/alpina/all/"><div class="IndexMarks__item-name">Alpina</div><div class="IndexMarks__item-count">2</div></a><a class="IndexMarks__item" h

In [11]:
#Пока набор данных получен в выде списка из пяти записей
len(marks_to_sale)

5

In [12]:
#Попробуем вынуть записи с ссылками
marks = list(map(lambda x: x.find_all('a', class_='IndexMarks__item'), marks_to_sale))
marks

[[<a class="IndexMarks__item" href="https://auto.ru/moskovskaya_oblast/cars/ac/all/"><div class="IndexMarks__item-name">AC</div><div class="IndexMarks__item-count">3</div></a>,
  <a class="IndexMarks__item" href="https://auto.ru/moskovskaya_oblast/cars/amc/all/"><div class="IndexMarks__item-name">AMC</div><div class="IndexMarks__item-count">2</div></a>,
  <a class="IndexMarks__item" href="https://auto.ru/moskovskaya_oblast/cars/acura/all/"><div class="IndexMarks__item-name">Acura</div><div class="IndexMarks__item-count">56</div></a>,
  <a class="IndexMarks__item" href="https://auto.ru/moskovskaya_oblast/cars/alfa_romeo/all/"><div class="IndexMarks__item-name">Alfa Romeo</div><div class="IndexMarks__item-count">33</div></a>,
  <a class="IndexMarks__item" href="https://auto.ru/moskovskaya_oblast/cars/alpina/all/"><div class="IndexMarks__item-name">Alpina</div><div class="IndexMarks__item-count">2</div></a>,
  <a class="IndexMarks__item" href="https://auto.ru/moskovskaya_oblast/cars/ariel

In [13]:
#Пока данные всё ещё в виде списка перечней
len(marks)
marks1 = list(chain(*marks))
print(len(marks1))

125


In [14]:
#Составим список из ссылок. При этом заменим  категорию "все" на "подержанные"
ref = []
for m in marks1:
    r = m.get('href')
    r = r.replace("/all/", "/used/")
    ref.append(r)
len(ref)

125

In [15]:
#Посмотрим, что же получилось
print(type(ref[121]), ref[121])

<class 'str'> https://auto.ru/moskovskaya_oblast/cars/moscvich/used/


In [16]:
ref_pages = []
ref_pages = Parallel(n_jobs=6)(delayed(ads_pages)(page_link) for page_link in tqdm(ref))

In [17]:
ref_pages_fin = list(chain(*ref_pages))
#Сохраним список ссылок на страницы в файл "про запас"
pd.Series(ref_pages_fin).to_csv('links_page.csv', index=False)


In [18]:
ref_pages = list(pd.read_csv('links_page.csv').values.flatten())

Получим список ссылок на все объявления о продаже подержанных машин

In [19]:
ads = []
ads = Parallel(n_jobs=10)(delayed(get_ad_url)(page_link) for page_link in tqdm(ref_pages))

In [20]:
#Преобразуем список, посмотрим, что получилось на случайно выбранном примере
ads_list = list(chain(*ads))
print(len(ads_list), ads_list[115])

51130 https://auto.ru/cars/used/sale/audi/a3/1101806677-2789db2e/


In [21]:
#Также сохраним этот список в файл
pd.Series(ads_list).to_csv('ads_links.csv', index=False)

In [6]:
#Извлечём список из файла. Этот код можно закомментировать, если скрипт выполняется подряд.
ads_list = list(pd.read_csv('ads_links.csv').values.flatten())

In [9]:
car_data_list = []
for ads in tqdm(ads_list):
    try:
        car_data_l = car_data(ads)
        car_data_list.append(car_data_l)
    except:
        pass
    

In [13]:
print(len(car_data_list),type(car_data_list[1]), car_data_list[11111])

51128 <class 'dict'> {'car_url': 'https://auto.ru/cars/used/sale/ford/v8/1088873606-f80ad2df/', 'parsing_unixtime': 1609854168, 'complectation_dict': '{"id":"0"}', 'equipment_dict': '{}', 'mileage': '99655', 'model_info': '{"code":"V8","name":"V8","ru_name":"в8","morphology":{}', 'model_name': 'V8', 'super_gen': '{"id":"21021631","year_from":1932,"year_to":1932,"price_segment":"MEDIUM"}', 'vendor': 'AMERICAN', 'sell_id': '1088873606', 'bodyType': 'седан', 'brand': 'FORD', 'color': 'чёрный', 'fuelType': 'бензин', 'modelDate': '1932', 'name': '3.6 MT (66 л.с.)', 'numberOfDoors': '2', 'productionDate': '1932', 'vehicleConfiguration': 'SEDAN MECHANICAL 3.6', 'vehicleTransmission': 'механическая', 'price': '2700000', 'priceCurrency': 'RUB', 'engineDisplacement': '3.6 LTR', 'enginePower': '66 N12', 'description': 'Форд А 1930 год. Купил в музее в США. На отличном ходу. Выезжаю несколько раз за лето. Из не оригинала генератор (но 6 вольт) и поворотники. Цвет красный. Обклеил черной пленкой и 

In [11]:
data = pd.DataFrame()
for car in car_data_list:
    try:
        data = data.append(car, ignore_index=True)
    except:
        pass
data.to_csv('car_auto_ru_info_train.csv', encoding = 'utf-8', index=False)